In [1]:
import numpy as np
np.set_printoptions(threshold = np.nan, linewidth = 115)
import pickle
import keras
from keras import models, optimizers, layers, regularizers, metrics, losses
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.core import Dense, Dropout, SpatialDropout2D, Activation
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import model_from_json, Sequential

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

IMG_SIZE = 256

# Save Comparison model
def save_model(model_name, hist_str, model_str):

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

Using TensorFlow backend.


In [ ]:
# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('1vAll_img_res_Infiltration_1st_half.p', 'rb'))
training_img_2 = pickle.load(open('1vAll_img_res_Infiltration_2nd_half.p', 'rb'))

training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('pickle_test_img.p', 'rb'))

labels_1 = pickle.load(open('1vAll_labels_res_Infiltration_1st_half.p', 'rb'))
labels_2 = pickle.load(open('1vAll_labels_res_Infiltration_2nd_half.p', 'rb'))

training_labels = np.append(labels_1, labels_2, axis = 0)

test_labels = pickle.load(open('test_labels.p', 'rb'))    

In [11]:
Infiltration_labels = training_labels[:, 8]
Infiltration_test_labels = test_labels[:, 8]
print(Infiltration_labels.shape)
print(len(Infiltration_test_labels))

(46184,)
11212


In [ ]:
# Metric Analysis
def true_accuracy(y_test, pred):
    
    pred = np.round_(pred)
    pred = pred.astype(dtype = 'uint8')
    
    ft = pred == y_test
    
#    acc = []
    
#    counter = 0
#    while counter < len(ft):
#        if sum(ft[counter]) < 15:
#            acc.append(0)
#            counter += 1
#        else:
#           acc.append(1)
#            counter += 1
           
    # Accuracy       
    Acc = (sum(ft)/len(ft))
    
   # print('\t Complete Label Accuracy: ', (Acc * 100), '%')
    
   # print('Sum of Fully Correct Predictions: ', sum(acc))
   # print('\t\t    Total Labels: ', len(acc))
    
    if Acc == 0:
        message = 'Feels Devastating (ﾉಥ益ಥ）ﾉ ┻━┻'
        
    elif Acc > 0 and Acc < 50:
        message = 'Feels Awful (੭ ˃̣̣̥ ㅂ˂̣̣̥)੭ु'
        
    elif Acc >= 50 and Acc < 60:
        message = 'Feels Bad (⌯˃̶᷄ ﹏ ˂̶᷄⌯)'
        
    elif Acc >= 60 and Acc < 70:
        message = 'Feels Meh... ┬─┬ノ(ಠ_ಠノ)'
    
    elif Acc >= 70 and Acc < 80:
        message = 'Feels Ok ʕ ·㉨·ʔ'
    
    elif Acc >= 80 and Acc < 90:
        message = 'Feels Better (^._.^)ﾉ'
        
    elif Acc >= 90 and Acc < 95:
        message = 'Feels Hopeful ( •́ ⍨ •̀)'
        
    elif Acc >= 95 and Acc < 98:
        message = 'Feels Good ヽ|･ω･|ゞ'
        
    elif Acc >= 98:
        message = 'Feels Great! ᕙ( * •̀ ᗜ •́ * )ᕗ'
        
    print('\n', message)
    
    return Acc

In [27]:
# Metric Analysis
def _1vAll_accuracy(y_test, pred):
    
    pred = np.round_(pred)
    pred = pred.astype(dtype = 'uint8')
    
    ft = pred == y_test
    
#    acc = []
    
#    counter = 0
#    while counter < len(ft):
#        if sum(ft[counter]) < 15:
#            acc.append(0)
#            counter += 1
#        else:
#           acc.append(1)
#            counter += 1
           
    # Accuracy       
    Acc = (sum(ft)/len(ft))
    
   # print('\t Complete Label Accuracy: ', (Acc * 100), '%')
    
   # print('Sum of Fully Correct Predictions: ', sum(acc))
   # print('\t\t    Total Labels: ', len(acc))
    accuracy = sum(Acc)/len(Acc) * 100
    
    return accuracy

In [ ]:
Atelectasis        = 112120 / 3790
Cardiomegaly       = 112120 / 972
Consolidation      = 112120 / 1188
Edema              = 112120 / 588
Effusion           = 112120 / 3617
Emphysema          = 112120 / 783
Fibrosis           = 112120 / 555
Hernia             = 112120 / 88
Infiltration       = 112120 / 8666
Mass               = 112120 / 2004
No_Finding         = 112120 / 53911
Nodule             = 112120 / 2478
Pleural_Thickening = 112120 / 983
Pneumonia          = 112120 / 283
Pneumothorax       = 112120 / 1942

print('       Atelectasis Weight: ', Atelectasis)       
print('      Cardiomegaly Weight: ', Cardiomegaly)      
print('     Consolidation Weight: ', Consolidation)     
print('             Edema Weight: ', Edema)             
print('          Effusion Weight: ', Effusion)          
print('         Emphysema Weight: ', Emphysema)         
print('          Fibrosis Weight: ', Fibrosis)          
print('            Hernia Weight: ', Hernia)            
print('      Infiltration Weight: ', Infiltration)      
print('              Mass Weight: ', Mass)              
print('        No_Finding Weight: ', No_Finding)        
print('            Nodule Weight: ', Nodule)            
print('Pleural_Thickening Weight: ', Pleural_Thickening)
print('         Pneumonia Weight: ', Pneumonia)         
print('      Pneumothorax Weight: ', Pneumothorax)

yung_class_weights = [{0: 1, 1: Atelectasis},
                      {0: 1, 1: Cardiomegaly},
                      {0: 1, 1: Consolidation},
                      {0: 1, 1: Edema},
                      {0: 1, 1: Effusion},
                      {0: 1, 1: Emphysema},
                      {0: 1, 1: Fibrosis},
                      {0: 1, 1: Hernia},
                      {0: 1, 1: Infiltration},
                      {0: 1, 1: Mass},
                      {0: 1, 1: No_Finding},
                      {0: 1, 1: Nodule},
                      {0: 1, 1: Pleural_Thickening},
                      {0: 1, 1: Pneumonia},
                      {0: 1, 1: Pneumothorax}]

In [8]:
Infiltration_class_weight = [{0: 1, 1: 12.9379183014078}]

### One Vs. All

In [10]:
# Infiltration One Vs. All

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

#model.summary()

# Last model: Infiltration_1vAll_20e
#model = load_model()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

model_obj = model.fit(training_img, Infiltration_labels, 
                      epochs = 20, 
                      validation_split = (1 / 8), 
                      batch_size = 128, verbose = 1, 
                      class_weight = Infiltration_class_weight)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(Infiltration_test_labels, Predictions)

history_str = 'Infiltration_1vAll_20e_history'
model_str   = 'Infiltration_1vAll_20e'
    
save_model(model_obj, history_str, model_str)

Train on 40411 samples, validate on 5773 samples
Epoch 1/20
40411/40411 [==============================] - 77s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9768 - val_acc: 0.6309
Epoch 2/20
40411/40411 [==============================] - 76s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9768 - val_acc: 0.6309
Epoch 3/20
40411/40411 [==============================] - 76s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9768 - val_acc: 0.6309
Epoch 4/20
40411/40411 [==============================] - 76s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9768 - val_acc: 0.6309
Epoch 5/20
40411/40411 [==============================] - 77s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9768 - val_acc: 0.6309
Epoch 6/20
40411/40411 [==============================] - 76s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9768 - val_acc: 0.6309
Epoch 7/20
40411/40411 [==============================] - 76s 2ms/step - loss: 82.6584 - acc: 0.6036 - val_loss: 76.9

In [25]:
accuracy = _1vAll_accuracy(Infiltration_test_labels, Predictions)

In [26]:
accuracy

84.43631823046735

### Baseline

In [ ]:
# v2: Classes weighted based on presence in data
# v3: Class Weights squared
# v4: Resampled data
# v5: No weights
# v6: Using OG Class Weights
# v7: Random Under Sample Data and using yung_class_weights

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='softmax'))

#model.summary()

# Last model: baseline_v7_40e
#model = load_model()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'categorical_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, 
                      epochs = 50, initial_epoch = 40, 
                      validation_split = (1 / 9), 
                      batch_size = 128, verbose = 1, 
                      class_weight = yung_class_weights)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_v7_50e_history'
model_str   = 'baseline_v7_50e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# Baseline Model for reference

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 128, verbose = 1)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_30_history'
model_str   = 'baseline_30'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Classes weighted based on presence in data

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1, class_weight=og_class_weights)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_v2_20e_history'
model_str   = 'baseline_v2_20e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Classes weighted based on presence in data
# v3: Class Weights squared

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1, class_weight=exp_weights)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_v3_20e_history'
model_str   = 'baseline_v3_20e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Classes weighted based on presence in data
# v3: Class Weights squared
# v4: Resampled data

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1, class_weight=exp_weights)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_v4_20e_history'
model_str   = 'baseline_v4_20e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Classes Weighted based on presence in data
# v3: Class weights squared
# v4: Resampled data
# v5: No weights

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_v5_20e_history'
model_str   = 'baseline_v5_20e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Classes weighted based on presence in data
# v3: Class Weights squared
# v4: Resampled data
# v5: No weights
# v6: Using OG Class Weights

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

#model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1, class_weight=og_class_weights)

Predictions = model.predict(test_img)
Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'baseline_v6_20e_history'
model_str   = 'baseline_v6_20e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
# v4: Added Batch Normalization to all layers
# v5: Using softmax final activation
# v6: Sigmoid final activation, yung_class_weights, 4 Blocks + Output, 32 Neurons in Dense Layer, SELU Activations,
#     Removed Batch Normalization, Added Dropout to Block 3

model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), 
                        kernel_regularizer=regularizers.l2(0.001), 
                        activation='selu', 
                        input_shape = (IMG_SIZE, IMG_SIZE, 1)))

model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(SpatialDropout2D(0.5))
#model.add(layers.BatchNormalization())

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
#model.add(layers.BatchNormalization())

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.375))
#model.add(layers.BatchNormalization())

# Block 4
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='selu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
#model.add(layers.BatchNormalization())

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='selu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, 
                      epochs = 30, validation_split = (1 / 9), 
                      batch_size = 128, verbose = 1, 
                      class_weight = yung_class_weights)

Predictions = model.predict(test_img)

Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'vanilla_reg_VGG_v6_30e_history'
model_str   = 'vanilla_reg_VGG_v6_30e'
    
save_model(model_obj, history_str, model_str)

### VGG_Doppleganger

In [ ]:
# Directly taken from VGG16 with 2D Spatial Dropout, Dropout, and fewer Dense layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(Activation('relu'))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, 
                      epochs = 30, validation_split = (1/9), 
                      batch_size = 128, verbose = 1)

history_str = 'VGG_Doppleganger_history'
model_str   = 'VGG_Doppleganger_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128, verbose = 1)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
# v3: Using Categorical Accuracy as metrics
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'VGG_Doppleganger_v3_30e'
    
save_model(model_obj, history_str, model_str)

### vanilla_VGG_Dropouts

In [ ]:
# Based on VGG with half reg'd layers and Dropout layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_history'
model_str   = 'vanilla_VGG_Dropouts_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v2_history'
model_str   = 'vanilla_VGG_Dropouts_v2_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v3_history'
model_str   = 'vanilla_VGG_Dropouts_v3_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v4_history'
model_str   = 'vanilla_VGG_Dropouts_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v5_history'
model_str   = 'vanilla_VGG_Dropouts_v5_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
# v6: Added Batch Normalization layers to each Block
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1, class_weight= class_weights )

history_str = 'vanilla_VGG_Dropouts_v6_history'
model_str   = 'vanilla_VGG_Dropouts_v6_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
Predictions = model.predict(test_img)
print(Predictions)
#Accuracy = true_accuracy(test_labels, Predictions)


### vanilla_reg_VGG

In [ ]:
# Modeled after VGGNet with half reg'd layers
vanilla_reg_VGG = models.Sequential()

# Block 1
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 2
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 3
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Output Block
vanilla_reg_VGG.add(layers.Flatten())
vanilla_reg_VGG.add(layers.Dense(64, activation='relu'))
vanilla_reg_VGG.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = vanilla_reg_VGG.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_history'
model_str   = 'vanilla_reg_VGG_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v2_history'
model_str   = 'vanilla_reg_VGG_v2_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v3_history'
model_str   = 'vanilla_reg_VGG_v3_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
# v4: Added Batch Normalization to all layers
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), 
              loss = 'binary_crossentropy', 
              metrics = ['categorical_accuracy'])

'''
model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

Predictions = model.predict(test_img)
Predictions = Predictions.astype(dtype = 'int32')

Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'vanilla_reg_VGG_v4_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)
'''

In [ ]:
model = Sequential()

model.add(SeparableConv2D(8, (3, 3), 
                          kernel_regularizer=regularizers.l2(0.001),
                          activation = 'elu', 
                          input_shape = (256, 256, 1)))
model.add(SeparableConv2D(8, (3, 3), 
                          kernel_regularizer=regularizers.l2(0.001),
                          activation = 'elu'))
model.add(SpatialDropout2D(0.5))
model.add(MaxPooling2D(2))

model.add(Conv2D(16, (3, 3), 
                 kernel_regularizer=regularizers.l2(0.001),
                 activation = 'elu'))
model.add(Conv2D(32, (3, 3), 
                 kernel_regularizer=regularizers.l2(0.001),
                 activation = 'elu'))
model.add(SpatialDropout2D(0.5))
model.add(MaxPooling2D(2))

model.add(layers.Flatten())
model.add(Dense(64, activation = 'elu'))
model.add(Dense(15, activation = 'sigmoid'))

model.summary()

model.compile(optimizer = optimizers.RMSprop(lr = 1e-3), 
              loss = 'binary_crossentropy',
              metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, 
                      epochs = 20, validation_split = (1/9), 
                      batch_size = 128, verbose = 1)

Predictions = model.predict(test_img)
Predictions = Predictions.astype(dtype = 'int32')

Accuracy = true_accuracy(test_labels, Predictions)

history_str = 'vanilla_reg_VGG_v4_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)